# If you're not running in Saturn Cloud, you need to install these libraries:

Make sure you use the latest versions

pip install -U transformers accelerate bitsandbytes

In [13]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [14]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         100G   47G   54G  47% /
tmpfs            64M     0   64M   0% /dev
tmpfs           7.7G     0  7.7G   0% /sys/fs/cgroup
/dev/nvme0n1p1  100G   47G   54G  47% /run
tmpfs            14G  4.0K   14G   1% /dev/shm
/dev/nvme2n1    2.0G  1.6M  1.9G   1% /home/jovyan
tmpfs            14G  120K   14G   1% /home/jovyan/.saturn
tmpfs            14G   12K   14G   1% /run/secrets/kubernetes.io/serviceaccount
tmpfs           7.7G   12K  7.7G   1% /proc/driver/nvidia
tmpfs           7.7G  4.5M  7.7G   1% /run/nvidia-persistenced/socket
tmpfs           7.7G     0  7.7G   0% /proc/acpi
tmpfs           7.7G     0  7.7G   0% /sys/firmware


In [15]:
import os
os.environ['HF_HOME'] = '/run/cache/'

In [16]:
import os
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Set the environment variable to use the desired cache directory
cache_dir = "/run/cache/transformers"
os.environ['HF_HOME'] = cache_dir

# Function to check if the model and tokenizer are already downloaded
def is_model_and_tokenizer_cached(cache_dir):
    model_files = [
        "pytorch_model.bin", 
        "config.json", 
        "tokenizer_config.json", 
        "spiece.model"
    ]
    return all(os.path.exists(os.path.join(cache_dir, "google/flan-t5-xl", file)) for file in model_files)

# Function to download the model and tokenizer if not cached
def get_tokenizer_and_model():
    if not is_model_and_tokenizer_cached(cache_dir):
        # Download the tokenizer and model, caching them to the specified directory
        tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl", cache_dir=cache_dir)
        model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", cache_dir=cache_dir, device_map="auto")
        print("Model and tokenizer downloaded and saved to cache directory.")
    else:
        # Load the tokenizer and model from the cache directory
        tokenizer = T5Tokenizer.from_pretrained(os.path.join(cache_dir, "google/flan-t5-xl"))
        model = T5ForConditionalGeneration.from_pretrained(os.path.join(cache_dir, "google/flan-t5-xl"), device_map="auto")
        print("Model and tokenizer loaded from cache directory.")
    return tokenizer, model

# Get the tokenizer and model
tokenizer, model = get_tokenizer_and_model()

# Your code to use the model and tokenizer here
# e.g., model.generate(...) or tokenizer(...)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.


Model and tokenizer downloaded and saved to cache directory.


In [17]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-09 08:13:09--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-09 08:13:09 (68.7 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [18]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [19]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [20]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [21]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids, )
    result = tokenizer.decode(outputs[0])
    return result

In [22]:
def llm(prompt, generate_params=None):
    if generate_params is None:
        generate_params = {}

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(
        input_ids,
        max_length=generate_params.get("max_length", 100),
        num_beams=generate_params.get("num_beams", 5),
        do_sample=generate_params.get("do_sample", False),
        temperature=generate_params.get("temperature", 1.0),
        top_k=generate_params.get("top_k", 50),
        top_p=generate_params.get("top_p", 0.95),
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

In [24]:
rag("I just discovered the course. Can I still join it?")

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."